In [1]:
# import polars as pl
import re
from tqdm import tqdm
import pandas as pd
import sqlite3 as db

combined_data_files = [
    r"data/combined_data_1.txt",
    r"data/combined_data_2.txt",
    r"data/combined_data_3.txt",
    r"data/combined_data_4.txt"

]

results = []
current_film = None
con = db.connect('netflix.db')

for file in combined_data_files[2:]:

    with open(file, "r") as file:
        lines = file.readlines()
        for line in tqdm(lines):
            if re.match(r"\d*:", line):
                current_film = int(line.strip()[:-1])
            else:
                vals = line.split(",")
                user = int(vals[0])
                rating = int(vals[1])
                date = vals[2]
                results.append([current_film, user, rating, date])
    print("Creating dataframe")
    data = pd.DataFrame(results, columns=["film", "user", "rating", "date"])
    print("Saving to DB")
    data.to_sql('netflix_data', con, if_exists='append', index=False)
    results = []

100%|██████████| 22605786/22605786 [00:40<00:00, 561613.40it/s]


Creating dataframe
Saving to DB


100%|██████████| 26851926/26851926 [00:51<00:00, 526002.14it/s]


Creating dataframe
Saving to DB


In [4]:

import sqlite3
import pandas as pd
import sqlite3 as db

con = sqlite3.connect('netflix.db')
# sql query that samples 1000 rows from the table
data = pd.read_sql_query("SELECT * FROM netflix_data ORDER BY RANDOM() LIMIT 100000", con)

In [5]:
con = db.connect('netflix_dev.db')
data.to_sql('netflix_data', con, if_exists='append', index=False)

100000

In [9]:
file_movies = "data/movie_titles.csv"

with open(file_movies, "r") as file:
    

print("Creating dataframe")
data = pd.DataFrame(results, columns=["film", "user", "rating", "date"])
print("Saving to DB")
data.to_sql('netflix_data', con, if_exists='append', index=False)
results = []

,film,user,rating,date
0,1,1488844,3,2005-09-06\n
1,1,822109,5,2005-05-13\n
2,1,885013,4,2005-10-19\n
3,1,30878,4,2005-12-26\n
4,1,823519,3,2004-05-03\n
...,...,...,...,...
24053759,4499,2591364,2,2005-02-16\n
24053760,4499,1791000,2,2005-02-10\n
24053761,4499,512536,5,2005-07-27\n
24053762,4499,988963,3,2005-12-20\n
